In [1]:
# Import the required modules
import pandas as pd
pd.set_option('display.max_columns', None)
import numpy as np

# Pre-Processing
from sklearn.preprocessing import StandardScaler, OneHotEncoder, OrdinalEncoder
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer
import pickle

# Models
from sklearn.neighbors import NearestNeighbors

# suppress warnings
import warnings
warnings.filterwarnings('ignore')

In [2]:
df = pd.read_csv("anime_ml.csv")
print(df.shape)
df.head()

(11903, 48)


,anime_id,name,episodes,rating,members,Action,Adventure,Cars,Comedy,Dementia,Demons,Drama,Fantasy,Game,Historical,Horror,Josei,Kids,Magic,MartialArts,Mecha,Military,Mystery,Parody,Police,Psychological,Romance,Samurai,School,SciFi,Seinen,Shoujo,ShoujoAi,Shounen,ShounenAi,SliceofLife,Space,Sports,SuperPower,Supernatural,Thriller,Vampire,Movie,Music,ONA,OVA,Special,TV
0,32281,Kimi no Na wa.,1,9.37,200630,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0
1,5114,Fullmetal Alchemist: Brotherhood,64,9.26,793665,1,1,0,0,0,0,1,1,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1
2,28977,Gintama,51,9.25,114262,1,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,1,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1
3,9253,Steins Gate,24,9.17,673572,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1
4,9969,Gintama',51,9.16,151266,1,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,1,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1


In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11903 entries, 0 to 11902
Data columns (total 48 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   anime_id       11903 non-null  int64  
 1   name           11903 non-null  object 
 2   episodes       11903 non-null  int64  
 3   rating         11903 non-null  float64
 4   members        11903 non-null  int64  
 5   Action         11903 non-null  int64  
 6   Adventure      11903 non-null  int64  
 7   Cars           11903 non-null  int64  
 8   Comedy         11903 non-null  int64  
 9   Dementia       11903 non-null  int64  
 10  Demons         11903 non-null  int64  
 11  Drama          11903 non-null  int64  
 12  Fantasy        11903 non-null  int64  
 13  Game           11903 non-null  int64  
 14  Historical     11903 non-null  int64  
 15  Horror         11903 non-null  int64  
 16  Josei          11903 non-null  int64  
 17  Kids           11903 non-null  int64  
 18  Magic 

In [4]:
df.describe()

,anime_id,episodes,rating,members,Action,Adventure,Cars,Comedy,Dementia,Demons,Drama,Fantasy,Game,Historical,Horror,Josei,Kids,Magic,MartialArts,Mecha,Military,Mystery,Parody,Police,Psychological,Romance,Samurai,School,SciFi,Seinen,Shoujo,ShoujoAi,Shounen,ShounenAi,SliceofLife,Space,Sports,SuperPower,Supernatural,Thriller,Vampire,Movie,Music,ONA,OVA,Special,TV
count,11903.000000,11903.000000,11903.000000,1.190300e+04,11903.000000,11903.000000,11903.000000,11903.000000,11903.000000,11903.000000,11903.000000,11903.000000,11903.000000,11903.000000,11903.000000,11903.000000,11903.000000,11903.000000,11903.000000,11903.000000,11903.000000,11903.000000,11903.000000,11903.000000,11903.000000,11903.000000,11903.000000,11903.000000,11903.000000,11903.000000,11903.000000,11903.000000,11903.000000,11903.000000,11903.000000,11903.000000,11903.000000,11903.000000,11903.000000,11903.000000,11903.000000,11903.000000,11903.000000,11903.000000,11903.000000,11903.000000,11903.000000
mean,13515.515080,12.426279,6.484570,1.844488e+04,0.233555,0.193901,0.005965,0.378728,0.019995,0.024364,0.166765,0.189028,0.014618,0.066454,0.030497,0.004369,0.130975,0.063849,0.021675,0.078216,0.035033,0.040830,0.033437,0.016382,0.019071,0.121314,0.012182,0.098715,0.171049,0.044695,0.049567,0.004537,0.141057,0.005377,0.099555,0.031589,0.044611,0.037554,0.084601,0.007225,0.008485,0.193397,0.040914,0.051164,0.273881,0.140217,0.300428
std,11174.446066,46.959892,1.016017,5.521622e+04,0.423110,0.395369,0.077005,0.485091,0.139989,0.154182,0.372781,0.391547,0.120024,0.249084,0.171956,0.065954,0.337388,0.244495,0.145627,0.268522,0.183871,0.197905,0.179782,0.126946,0.136780,0.326505,0.109701,0.298291,0.376568,0.206641,0.217058,0.067205,0.348095,0.073132,0.299418,0.174910,0.206456,0.190122,0.278298,0.084696,0.091728,0.394978,0.198099,0.220341,0.445967,0.347226,0.458464
min,1.000000,1.000000,1.670000,5.000000e+00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,3361.500000,1.000000,5.900000,2.340000e+02,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,9885.000000,2.000000,6.570000,1.596000e+03,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
75%,23485.000000,12.000000,7.180000,9.830000e+03,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,1.000000
max,34527.000000,1818.000000,10.000000,1.013917e+06,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,

In [5]:
def model_fit(anime_length):
    # Step 1: Load the dataset
    df = pd.read_csv("anime_ml.csv")
    
    # Remove any rows with missing values and reset the index
    df = df.dropna(how="any").reset_index(drop=True)

    
    # Remove duplicate anime based on track_id and reset index again
    df = df.drop_duplicates(subset=["name"]).reset_index(drop=True)

 
    # Step 2: Define the columns for metadata and features
    meta_cols = ["anime_id","name"]  # Metadata columns to keep anime info
    feature_cols = ['episodes', 'rating', 'members', 'Action', 'Adventure', 'Cars',
       'Comedy', 'Dementia', 'Demons', 'Drama', 'Fantasy', 'Game',
       'Historical', 'Horror', 'Josei', 'Kids', 'Magic', 'MartialArts',
       'Mecha', 'Military', 'Mystery', 'Parody', 'Police',
       'Psychological', 'Romance', 'Samurai', 'School', 'SciFi', 'Seinen',
       'Shoujo', 'ShoujoAi', 'Shounen', 'ShounenAi', 'SliceofLife', 'Space',
       'Sports', 'SuperPower', 'Supernatural', 'Thriller', 'Vampire', 'Movie',
       'Music', 'ONA', 'OVA', 'Special', 'TV']

    # Step 3: Define preprocessing steps for the data

    # Preprocessing for numeric features: fill missing values with the mean and standardize the values
    numeric_features = []
    numeric_transformer = Pipeline(steps=[
        ('imputer', SimpleImputer(strategy='mean')),  # Fill missing values with the mean
        ('scaler', StandardScaler())])  # Standardize features

    # Preprocessing for binary features (if any): fill missing values with most frequent and label encode
    binary_features = []
    binary_transformer = Pipeline(steps=[
        ('imputer', SimpleImputer(strategy='most_frequent', missing_values=pd.NA)),  # Fill missing values
        ('label', OrdinalEncoder())])  # Convert binary features to ordinal values

    # Preprocessing for categorical features: fill missing values and apply one-hot encoding
    categorical_features = []
    categorical_transformer = Pipeline(steps=[
        ('imputer', SimpleImputer(strategy='most_frequent', missing_values=pd.NA)),  # Fill missing values
        ('onehot', OneHotEncoder(handle_unknown='ignore'))])  # Apply one-hot encoding

    # Combine all preprocessing steps into a single preprocessor
    preprocessor = ColumnTransformer(
        transformers=[
            ('num', numeric_transformer, numeric_features),  # Numeric features preprocessing
            ('binary', binary_transformer, binary_features),  # Binary features preprocessing (if any)
            ('cat', categorical_transformer, categorical_features)])  # Categorical features preprocessing


    # Step 5: Prepare the feature matrix for the Nearest Neighbors model
    X = df.loc[:, feature_cols]  # Select the feature columns

    # Apply preprocessing to the feature matrix
    # preprocessor.fit(X)  # Fit the preprocessor to the feature matrix
    # X_preprocessed = preprocessor.transform(X)  # Transform the feature matrix

    # Step 6: Initialize the Nearest Neighbors model
    # # define the number of nearest neighbors to consider and use cosine similarity as the metric
    k = anime_length
    model1 = NearestNeighbors(n_neighbors=k, metric="cosine")

    # Fit the Nearest Neighbors model to the preprocessed data
    # model1.fit(X_preprocessed)
    model1.fit(X)
    # Save the model to a pickle file
    
    with open('anime_name.pkl', 'wb') as model_file:
        pickle.dump(model1, model_file)

def make_recommendation(anime_model, anime_name):
    
    # Step 1: Load the dataset
    df = pd.read_csv("anime_ml.csv")
    
   
    # Load Pickle file
    filename = anime_model
    model1 = pickle.load(open(filename, 'rb')) 

    feature_cols = ['episodes', 'rating', 'members', 'Action', 'Adventure', 'Cars',
       'Comedy', 'Dementia', 'Demons', 'Drama', 'Fantasy', 'Game',
       'Historical', 'Horror', 'Josei', 'Kids', 'Magic', 'MartialArts',
       'Mecha', 'Military','Mystery', 'Parody', 'Police',
       'Psychological', 'Romance', 'Samurai', 'School', 'SciFi', 'Seinen',
       'Shoujo', 'ShoujoAi', 'Shounen', 'ShounenAi', 'SliceofLife', 'Space',
       'Sports', 'SuperPower', 'Supernatural', 'Thriller', 'Vampire', 'Movie',
       'Music', 'ONA', 'OVA', 'Special', 'TV']
    
        # Step 4: Get the target track's ID based on track name and artist
    # Select the most popular track by track_name and track_artist combination
    anime_id = df.loc[(df.name == anime_name)] \
                .sort_values(by="rating", ascending=False).anime_id.values[0]
    # anime_id = 22319
    
    # Step 7: Extract features of the anime
    anime_features = df.loc[df.anime_id == anime_id, feature_cols]  # Get the feature vector for the target anime
    # anime_features_preprocessed = preprocessor.transform(anime_features)  # Preprocess the target track features
    # print(anime_features)
    # Step 8: Find the nearest neighbors (anime most similar to the target selection)
    # distances, indices = model1.kneighbors(anime_features_preprocessed)  # Get distances and indices of neighbors
    distances, indices = model1.kneighbors(anime_features)  # Get distances and indices of neighbors

    # Step 9: Retrieve the metadata of the recommended tracks
    animes = df.iloc[indices[0]]  # Select anime corresponding to the nearest neighbors
    animes["distance"] = distances[0]  # Add the distance of each neighbor as a new column

    # Step 10: Filter the columns for the final output
    cols = animes.columns # you can explicitly choose to return specific columns here
    animes = animes.loc[:, cols]  # Keep the relevant columns
    animes = animes.sort_values(by="distance")  # Sort the tracks by their distance (most similar first)

    # Step 11: Return the recommended tracks as a list of dictionaries
    return animes.to_dict(orient="records")

In [8]:
model_fit(11899)

In [9]:
 # User input
anime_model = "anime_name.pkl"
anime_name = "Naruto"


response = make_recommendation(anime_model, anime_name) # used for the flask app, returns JSON

# to test
df1 = pd.DataFrame(response)
df1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11899 entries, 0 to 11898
Data columns (total 49 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   anime_id       11899 non-null  int64  
 1   name           11899 non-null  object 
 2   episodes       11899 non-null  int64  
 3   rating         11899 non-null  float64
 4   members        11899 non-null  int64  
 5   Action         11899 non-null  int64  
 6   Adventure      11899 non-null  int64  
 7   Cars           11899 non-null  int64  
 8   Comedy         11899 non-null  int64  
 9   Dementia       11899 non-null  int64  
 10  Demons         11899 non-null  int64  
 11  Drama          11899 non-null  int64  
 12  Fantasy        11899 non-null  int64  
 13  Game           11899 non-null  int64  
 14  Historical     11899 non-null  int64  
 15  Horror         11899 non-null  int64  
 16  Josei          11899 non-null  int64  
 17  Kids           11899 non-null  int64  
 18  Magic 

In [11]:
df.head(15)

,anime_id,name,episodes,rating,members,Action,Adventure,Cars,Comedy,Dementia,Demons,Drama,Fantasy,Game,Historical,Horror,Josei,Kids,Magic,MartialArts,Mecha,Military,Mystery,Parody,Police,Psychological,Romance,Samurai,School,SciFi,Seinen,Shoujo,ShoujoAi,Shounen,ShounenAi,SliceofLife,Space,Sports,SuperPower,Supernatural,Thriller,Vampire,Movie,Music,ONA,OVA,Special,TV
0,32281,Kimi no Na wa.,1,9.37,200630,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0
1,5114,Fullmetal Alchemist: Brotherhood,64,9.26,793665,1,1,0,0,0,0,1,1,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1
2,28977,Gintama,51,9.25,114262,1,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,1,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1
3,9253,Steins Gate,24,9.17,673572,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1
4,9969,Gintama',51,9.16,151266,1,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,1,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1
5,32935,Haikyuu!!: Karasuno Koukou VS Shiratorizawa Ga...,10,9.15,93351,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,1
6,11061,Hunter x Hunter (2011),148,9.13,425855,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,1
7,820,Ginga Eiyuu Densetsu,110,9.11,80679,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0
8,15335,Gintama Movie: Kanketsu-hen - Yorozuya yo Eien...,1,9.10,72534,1,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,1,0,1,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0
9,15417,Gintama': Enchousen,13,9.11,81109,1,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,1,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1
